# Qonto - Releve de compte augmente
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Qonto/Qonto_Releve_de_compte_augmente.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #qonto #bank #statement #naas_drivers #notification #emailbuilder #asset #scheduler #naas

Recevez un relevé de compte augmenté par email gratuitement, chaque semaine, grâce à un template Naas.ai (moteur de données opensource, 100 crédits offert par mois). 
<br><br>
-Durée de l'installation = 5 minutes<br>
-Support d'installation = Guide vidéo<br>
-Niveau = Facile<br>

## Input
Dans cette section, vous trouverez les informations à configurer pour que ce notebook puisse accéder à vos données via l'API Qonto.

### Import des librairies
Les libraries sont des outils créé dans le language Python qui permettent le fonctionnement du notebook. Aucune action nécessaire. 

In [ ]:
from naas_drivers import qonto
from datetime import datetime, timedelta
import pandas as pd
import naas

### Configuration des accès API
👇 Veuillez saisir ci-dessous, entre les guillemets, votre identifiant et votre clé secrète récupérés sur la plateforme Qonto.
<a href='https://www.notion.so/naas-official/Qonto-driver-Get-your-credentials-0cc97828b4e7467c8bfbcf704a77e5f4'>Comment récupérer ces accès API ?</a>

In [ ]:
QONTO_USER_ID = 'YOUR_USER_ID'
QONTO_SECRET_KEY = 'YOUR_SECRET_KEY'

### Configuration de l'email
👇 Veuillez saisir ci-dessous, entre les guillemets, le destinataire de l'email (si vous avez plusieurs destinataires, separez d'une virgule les emails en conservants les guillemets)<br>
Vous pouvez aussi changer l'objet de l'email (configuration avancée)

In [ ]:
# Destinataire
EMAIL_TO = ["Votre_adresse_email_ici"]

# Objet de l'email
EMAIL_SUBJECT = f"🏛️ Qonto - Votre relevé de compte augmenté du {datetime.now().strftime('%d/%m/%Y')}"

### Configuration de la période de l'analyse
👇 Veuillez saisir ci-dessous, entre les guillemets, la date de début (et la date de fin de votre analyse.

In [ ]:
# Date de début au format AAAA-MM-JJ
DATE_FROM = "2021-01-01"

# Date de fin au format AAAA-MM-JJ (par defaut, c'est la date d'aujoud'hui qui est selectionnée)
DATE_TO = datetime.now().strftime("%Y-%m-%d")

# Nombre de jours de récupération des dernières transactions (doit être un chiffre négatif)
LAST_TRANSACTIONS = -7

### Configuration de l'automatisation
Grâce à la formule ci-dessous, le notebook se lancera tous les lundis à 8h.<br>
Si vous souhaitez modifier la fréquence d'envoi, vous devez modifier la synthaxe entre guillemets en  <a href="https://crontab.guru/">suivant la documentation officielle CRON</a> (standard internationnal pour la programmation de tâches automatisées)

In [ ]:
naas.scheduler.add(cron="0 8 * * 1")

**Configuration des noms de fichiers (avancé)**

In [ ]:
GRAPH_FILE = "graph_account_statement.html"
GRAPH_IMG = "graph_account_statement.jpeg"
TABLE_FILE = "account_statement.xlsx"

## Model

### Récupération du relevé de compte consolidé

In [ ]:
# Colonne to consolidate (DATE already included), if empty return only DATE, AMOUNT, POSITION
to_conso = ["TRANSACTION_ID",
            "LABEL",
            "OPERATION_TYPE"]

df_statement = qonto.connect(QONTO_USER_ID, QONTO_SECRET_KEY).statement.consolidated(to_conso=to_conso,
                                                                                     date_from=DATE_FROM,
                                                                                     date_to=DATE_TO)
df_statement

### Création du graphique "Evolution de la Trésorerie"

In [ ]:
barline = qonto.connect(QONTO_USER_ID, QONTO_SECRET_KEY).statement.barline(date_from=DATE_FROM,
                                                                           date_to=DATE_TO)
barline

### Récupération des opérations par type

In [ ]:
cash_summary = qonto.connect(QONTO_USER_ID, QONTO_SECRET_KEY).statement.summary(date_from=DATE_FROM,
                                                                                date_to=DATE_TO)
cash_summary

### Récupération des dernières transactions

In [ ]:
df_last = qonto.connect(QONTO_USER_ID, QONTO_SECRET_KEY).statement.transactions(date_from=LAST_TRANSACTIONS,
                                                                                date_to=DATE_TO)
df_last

### Calcule du solde courant

In [ ]:
current_position = round(df_statement['POSITION'].tolist()[-1], 2)
current_position

## Output

### Sauvegarde du relevé de compte et partage du fichier Excel

In [ ]:
df_statement.to_excel(TABLE_FILE)
statement_link = naas.asset.add(TABLE_FILE)

### Sauvegarde et partage du graphique en tant qu'image et page html

In [ ]:
# Image
barline.write_image(GRAPH_IMG)
graph_img = naas.asset.add(GRAPH_IMG)

# HTML
barline.write_html(GRAPH_FILE)
params = {"inline": True}
graph_link = naas.asset.add(GRAPH_FILE, params=params)

### Creation de l'email

In [ ]:
email_content = qonto.connect(QONTO_USER_ID, QONTO_SECRET_KEY).statement.email(DATE_FROM,
                                                                               DATE_TO,
                                                                               current_position,
                                                                               graph_img,
                                                                               graph_link,
                                                                               cash_summary,
                                                                               LAST_TRANSACTIONS,
                                                                               df_last,
                                                                               statement_link)

### Envoi de l'email

In [ ]:
naas.notification.send(EMAIL_TO,
                       EMAIL_SUBJECT,
                       email_content)